In [ ]:
# Importer les bibliothèques nécessaires
import pandas as pd

# Étape 1 : Charger les bases ETRETAT_PM et ETRETAT_PP
# Load ETRETAT_PM and ETRETAT_PP datasets
etretat_pm = pd.read_csv('DE_ETRETAT_RENTABILITE_SIEGE_PM_2024_11_14$22_41_12.csv', sep=';')
etretat_pp = pd.read_csv('DE_ETRETAT_RENTABILITE_SIEGE_PP_2024_11_14$23_07_34.csv', sep=';')

# Étape 2 : Garder uniquement les colonnes nécessaires
# Keep only relevant columns
columns_to_keep = ['cd_tiers', 'CIR_RWA', 'CIR_CNR_FLUX']
etretat_pm = etretat_pm[columns_to_keep].copy()
etretat_pp = etretat_pp[columns_to_keep].copy()

# Étape 3 : Renommer les colonnes pour chaque base
# Rename columns for each dataset
etretat_pm.rename(columns={
    'CIR_RWA': 'RWA_CNR_PM',
    'CIR_CNR_FLUX': 'CNR_CNR_PM'
}, inplace=True)

etretat_pp.rename(columns={
    'CIR_RWA': 'RWA_CNR_PP',
    'CIR_CNR_FLUX': 'CNR_CNR_PP'
}, inplace=True)

# Ajouter un préfixe pour différencier les deux datasets
# Add prefix to differentiate datasets
etretat_pm.rename(columns=lambda x: f"PM_{x}" if x != 'cd_tiers' else x, inplace=True)
etretat_pp.rename(columns=lambda x: f"PP_{x}" if x != 'cd_tiers' else x, inplace=True)

# Étape 4 : Groupby sur 'cd_tiers' pour obtenir une ligne unique par client
# Groupby on 'cd_tiers' to have one unique row per client
etretat_pm_grouped = etretat_pm.groupby('cd_tiers').sum().reset_index()
etretat_pp_grouped = etretat_pp.groupby('cd_tiers').sum().reset_index()

# Étape 5 : Importer la base Base_VPVD_PNB
# Load the base Base_VPVD_PNB
base_vpvd_pnb = pd.read_csv('Base_VPVD_PNB.csv', sep=';')

# Étape 6 : Merge des bases avec Base_VPVD_PNB
# Merge the datasets with Base_VPVD_PNB
# Merge avec ETRETAT_PP
# Merge with ETRETAT_PP
base_merged = base_vpvd_pnb.merge(
    etretat_pp_grouped,
    left_on='PM_CD_TIERS',
    right_on='cd_tiers',
    how='left'
)

# Merge avec ETRETAT_PM
# Merge with ETRETAT_PM
base_merged = base_merged.merge(
    etretat_pm_grouped,
    left_on='PP_CD_TIERS',
    right_on='cd_tiers',
    how='left',
    suffixes=('_pp', '_pm')
)

# Étape 7 : Nettoyer les colonnes inutiles
# Drop unnecessary columns
base_merged.drop(columns=['cd_tiers_pp', 'cd_tiers_pm'], inplace=True)

# Étape 8 : Exporter le fichier final
# Export the final file
base_merged.to_csv('Base_VPVD_PNB_Final.csv', sep=';', index=False)

print("Script terminé avec succès, fichier exporté.")


In [ ]:
import pandas as pd
import numpy as np

# Charger les données
# Base_VPVD_DAT_R = pd.read_csv("chemin_vers_votre_fichier.csv")  # Exemple d'import

# Étape 1: Calcul de `SUM_RWA_PM`
Base_VPVD_DAT_R["SUM_RWA_PM"] = Base_VPVD_DAT_R.groupby("PP_CD_TIERS")["PM_CIR_RWA"].transform("sum")

# Étape 2: Calcul de `RATIO DE RWA`
def calculate_ratio(row):
    if row["NB_PM par PP"] == 1:
        return 1
    elif pd.isna(row["PP_CD_TIERS"]):
        return np.nan
    else:
        return row["PM_CIR_RWA"] / row["SUM_RWA_PM"]

Base_VPVD_DAT_R["RATIO_DE_RWA"] = Base_VPVD_DAT_R.apply(calculate_ratio, axis=1)

# Étape 3: Calcul de `ALLOCATION RWA PP`
Base_VPVD_DAT_R["ALLOCATION_RWA_PP"] = Base_VPVD_DAT_R["PM_CIR_RWA"] * Base_VPVD_DAT_R["RATIO_DE_RWA"]

# Étape 4: Calcul de `SOMME RWA VD par PM`
Base_VPVD_DAT_R["SOMME_RWA_VD_par_PM"] = Base_VPVD_DAT_R.groupby("PM_CD_TIERS")["ALLOCATION_RWA_PP"].transform("sum")

# Exporter le résultat (si nécessaire)
# Base_VPVD_DAT_R.to_csv("chemin_vers_le_fichier_de_sortie.csv", index=False)

# Affichage des premières lignes pour vérifier le résultat
print(Base_VPVD_DAT_R.head())
